In [ ]:
!pip install sklearn-genetic
!pip install pycaret-nightly

In [ ]:
import pandas as pd 
import numpy as np 
import sklearn
from google.colab import files, drive, data_table
from google.colab.data_table import *
from lightgbm import *
from sklearn.ensemble import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from xgboost import *
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.tree import *
from sklearn.utils import *
from yellowbrick.model_selection import * 
from genetic_selection import GeneticSelectionCV
from yellowbrick.classifier import * 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as scs
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import chi2
mcc = make_scorer(matthews_corrcoef)

In [ ]:
D = pd.read_csv("/content/drive/MyDrive/Reddy_NLP/test_cleaned.csv")
#D.describe()
#D.info()

In [ ]:


bins = [D.VehBCost.min(),5000, 7000, 8000, 10000, D.VehBCost.max()]
D['VehBCost'] = pd.cut(D['VehBCost'], bins)

bins = [D.WarrantyCost.min(),700, 1200, 1800, 2600, D.WarrantyCost.max()]
D['WarrantyCost'] = pd.cut(D['WarrantyCost'], bins)

bins = [D.VehOdo.min(),40000, 60000, 80000, 100000, D.VehOdo.max()]
D['VehOdo'] = pd.cut(D['VehOdo'], bins)

D['VehicleAge'] = pd.cut(D['VehicleAge'], [0, 2, 4, 6, 8, 10], right=False)

bins = [D.ARCP.min(), 3000, 6000, 8000, 10000, D.ARCP.max()]
D['ARCP'] = pd.cut(D['ARCP'], bins)

bins = [D.AAAP.min() - 1, 3000, 6000, 8000, 10000, D.AAAP.max()]
D['AAAP'] = pd.cut(D['AAAP'], bins)

bins = [D.AACP.min() - 1, 3000, 6000, 8000, 10000, D.AACP.max()]
D['AACP'] = pd.cut(D['AACP'], bins)


bins = [D.ARAP.min() - 1, 3000, 6000, 8000, 10000, D.ARAP.max()]
D['ARAP'] = pd.cut(D['ARAP'], bins)

bins = [D.CAAP.min() - 1, 3000, 6000, 8000, 10000, D.CAAP.max()]
D['CAAP'] = pd.cut(D['CAAP'], bins)

bins = [D.CACP.min() - 1, 3000, 6000, 8000, 10000, D.CACP.max()]
D['CACP'] = pd.cut(D['CACP'], bins)

bins = [D.CRAP.min() - 1, 3000, 6000, 8000, 10000, D.CRAP.max()]
D['CRAP'] = pd.cut(D['CRAP'], bins)

bins = [D.CRCP.min() - 1, 3000, 6000, 8000, 10000, D.CRCP.max()]
D['CRCP'] = pd.cut(D['CRCP'], bins)



In [ ]:
 
label_encoder = preprocessing.LabelEncoder()

c1_columns = list(D.select_dtypes(include='object').columns)
c2_columns = list(D.select_dtypes(include='category').columns)
cat_columns = c1_columns+c2_columns

for c in cat_columns:
  D[c] = label_encoder.fit_transform(D[c])

In [ ]:

D = D.drop(["Unnamed: 0","RefId","BYRNO",'PurchDate','TopThreeAmericanName','IsOnlineSale','Region','IsBase','AuctionAVG','RetailAVG','Trend'], axis=1)

In [ ]:
allfeats = list(D.columns)
allfeats.remove('IsBadBuy')

In [ ]:
D.iloc[:,-1] = D.iloc[:,-1].astype(np.int64)
X = D[allfeats] ; y = D["IsBadBuy"].values

In [ ]:

estimator = LogisticRegression(solver = "liblinear", C = 1, tol = 1, fit_intercept = True)

from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 
rkf = RepeatedStratifiedKFold(n_repeats = 5, n_splits = 5)
for i in range(2,22):
  
  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = mcc,
                                max_features = i,
                                n_population = 50,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 5,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.1,
                                #tournament_size = 3,
                                n_gen_no_change=5,
                                caching=True,
                                n_jobs=-1)
  selector = selector.fit(D[allfeats], y)
  genfeats = D[allfeats].columns[selector.support_]
  genfeats = list(genfeats)
  print("Chosen Feats:  ", genfeats)

  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore


Chosen Feats:   ['VehicleAge']
Chosen Feats:   ['VehicleAge', 'VehOdo']
Chosen Feats:   ['VehicleAge', 'VehOdo', 'ARCP']
Chosen Feats:   ['VehicleAge', 'VehOdo', 'ARCP', 'VehBCost']
Chosen Feats:   ['VehicleAge', 'VehOdo', 'ARCP', 'VehBCost', 'WarrantyCost']


In [ ]:

report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]
DataTable(report)
print(ga_feats)

['VehicleAge', 'VehOdo', 'ARCP', 'VehBCost', 'WarrantyCost']


In [ ]:

def categories(series):
    return range(int(series.min()), int(series.max()) + 1)


def chi_square_of_df_cols(df, col1, col2):
    df_col1, df_col2 = df[col1], df[col2]

    result = [[sum((df_col1 == cat1) & (df_col2 == cat2))
               for cat2 in categories(df_col2)]
              for cat1 in categories(df_col1)]

    return scs.chi2_contingency(result)

useful_col = []
alpha = 0.05
for c in list(D.columns):
      stat, p, dof, expected = chi_square_of_df_cols(D, c, 'IsBadBuy')
      if p<alpha:
        useful_col.append(c)

print(useful_col)



['VehicleAge', 'VehOdo', 'ARCP', 'ARAP', 'CRCP', 'VehBCost', 'WarrantyCost']


In [ ]:
D.to_csv('input_to_model.csv')